# RNA-Seq data analysis Airway Organoid - Pseudomonas Project. 
## Heatmaps of gene-regulation for specific gene-sets 


In [43]:
import pandas as pd 
import os 
import altair as alt
from typing import Literal


count_per_million = "/data/scratch/kvalem/projects/2023/pseudomonas_aeruginosa/tables/log2_cpm_counts_plus_one.tsv"
deseq2_results = "/data/projects/2023/Pseudomonas_aeruginosa/20_deseq2icbi/01_deseq2/hAO_inf_vs_hAO_ctrl/"


In [44]:
count_mat = pd.read_csv(count_per_million,index_col=0 ) #LOAD LAREADY NORMALIZED COUNT

In [45]:
count_mat[['gene_ensemlb','transcript']] = count_mat['gene_id'].str.split('.', expand=True)
count_mat = count_mat[count_mat.columns.drop(list(count_mat.filter(regex="COPD")))]

### Merge duplicated transcripts with same gene name  and calculate the mean for each columns 

In [46]:
col_dict  = {}
for i in count_mat.columns[2:-2]:
    col_dict[i] = "mean"

In [47]:
count_mat_mean = count_mat.groupby(count_mat['gene_ensemlb']).aggregate(col_dict)

### Define gene sets of interest

#### GSEA BIOLOGICAL PRODUCTS 

In [48]:
gsea_bp = "hAO_inf_hAO_ctrl_GSEA_GO_BP.tsv"
path = os.path.join(deseq2_results,gsea_bp)
bioprod_gsea = pd.read_csv(path , sep = "\t")

In [49]:
geneset_of_interest_name = ["response to bacterium",
                           "response to lipopolysaccharide",
                           "regulation of defence response",
                           "nitric oxide metabolic process",
                           "reactive oxygen species metabolic process",
                           "antimicrobial humoral response",
                           "antimicrobial humoral immune response mediated by antimicrobial peptide",
                           #"Signaling by Interleukins", from REACTOME 
                           "cilium assembly",
                           "cilium organization",
                           "cilium movement"]

In [125]:
for i in range(len(geneset_of_interest_name)):
    gene_set = bioprod_gsea[bioprod_gsea["Description"] == geneset_of_interest_name[i]]["core_enrichment"]
    gene_set_updated_list = [item.replace('/', ',') for item in gene_set.to_list()]
    if len(gene_set_updated_list)>0:
        gene_set_updated_list = gene_set_updated_list[0].split(',')
        print(gene_set_updated_list)
    else:
        print("Empty gene set: "+ str(geneset_of_interest_name[i]))


['SOD2', 'AKAP12', 'CYP1A1', 'TNIP3', 'DEFB4A', 'TNFRSF1B', 'CSF3', 'TNIP1', 'PI3', 'LYN', 'CCL2', 'IRAK3', 'SLC7A5', 'RIPK2', 'TNIP2', 'GCH1', 'TNFRSF11A', 'S100A9', 'CXCL8', 'IL36G', 'BMP2', 'LITAF', 'NOS2', 'PTAFR', 'SRC', 'PTGS2', 'PGLYRP4', 'IRAK2', 'CXCL6', 'NFKB1', 'CXCL3', 'KMO', 'CX3CL1', 'ARG2', 'TLR2', 'NFKBIB', 'ADAM17', 'PGLYRP2', 'RELA', 'TGFB1', 'MAPKAPK2', 'IFNAR1', 'CCL20', 'BPIFA2', 'CXCL5', 'MAP2K3', 'SBNO2', 'JAK2', 'PRG2', 'GJB2', 'VGF', 'BAIAP2L1', 'CSF1', 'AKAP8', 'UGT1A1', 'TNFAIP3', 'SMAD6', 'ZC3H12A', 'S100A8', 'TMF1', 'IL23A', 'DHX15', 'TNF', 'CYRIB', 'ANKRD17', 'PGLYRP3', 'SELENOS', 'S100A12', 'IRF3', 'MAPK14', 'GBP2', 'H2BC21', 'TNFRSF14', 'RGS1', 'NOTCH1', 'RARRES2', 'SIRPA', 'NOD2', 'CD274', 'ASS1', 'S100A7', 'MAPK8', 'AKIRIN2', 'IL1A', 'BNIP3', 'CXCL2', 'IDO1', 'CITED1', 'NFKBIA', 'CXCL1', 'GJB6', 'TENT5A', 'TICAM2', 'TBK1', 'C15orf48', 'BCL10', 'DMBT1', 'RAB1A']
['SOD2', 'AKAP12', 'CYP1A1', 'TNIP3', 'TNFRSF1B', 'CSF3', 'TNIP1', 'LYN', 'CCL2', 'IRAK3', '

In [129]:
count_mat_subset_dict={}
for i in range(len(geneset_of_interest_name)):
    gene_set = bioprod_gsea[bioprod_gsea["Description"] == geneset_of_interest_name[i]]["core_enrichment"]
    gene_set_updated_list = [item.replace('/', ',') for item in gene_set.to_list()]
    if len(gene_set_updated_list)>0:
        gene_set_updated_list = gene_set_updated_list[0].split(',')
        count_mat_subset = pd.DataFrame()
        for gene in gene_set_updated_list:
            if gene in list(count_mat["gene_name"]):
                count_mat_subset = pd.concat([count_mat_subset,count_mat[count_mat["gene_name"]== gene]])
                del count_mat_subset["gene_ensemlb"]
                del count_mat_subset['transcript']
                del count_mat_subset['gene_id']
                count_mat_subset_dict[geneset_of_interest_name[i]]=count_mat_subset
    else:
        print("Empty gene set: "+ str(geneset_of_interest_name[i]))
   
            

Empty gene set: regulation of defence response


In [157]:
count_mat_subset.head()

,gene_name,hAO_r1_ctrl,hAO_r1_inf,hAO_r2_ctrl,hAO_r2_inf,hAO_r3_ctrl,hAO_r3_inf,hAO_r4_ctrl,hAO_r4_inf
4001,SOD2,7.541443,9.833153,8.039504,10.108487,7.694543,10.013012,7.668532,9.631477
6380,AKAP12,5.436937,6.946795,4.747114,6.408801,4.541001,6.579520,4.829239,6.410048
7969,CYP1A1,7.145175,8.826284,5.635661,8.829776,6.150972,8.585044,5.836614,8.115452
685,TNIP3,3.171267,5.275863,4.175268,5.808995,4.085050,6.020813,3.891125,5.899015
13119,DEFB4A,5.992566,7.392839,5.048574,6.827780,5.279793,6.718589,5.330398,7.179204


In [158]:
path_out = "/data/scratch/kvalem/projects/2023/pseudomonas_aeruginosa/tables/"
count_mat_subset.to_csv(path_out+"count_mat_heatmap.csv")

#### GSEA MOLECULAR FUNCTION

#### GSEA KEGG

#### GSEA REACTOME

#### GSEA WIKIPATHWAY